In [1]:
!pip install -q gradio plotly scikit-learn pandas numpy matplotlib

In [2]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Loading data
df = pd.read_csv(
    "https://raw.githubusercontent.com/Ak-270704/ATM-Cash-Demand-ML-Minor-Project/main/ATM%20Cash%20Demand%20Forecast%20for%20BankServe%20(2).csv",
    parse_dates=['date']
)

print("Dataset loaded successfully!")
print(f"Shape: {df.shape}")
df.head()

Dataset loaded successfully!
Shape: (1000, 9)


,atm_id,date,location_type,nearby_population,income_level,holiday_flag,local_event_flag,season,forecasted_cash_demand
0,1096,2023-01-01,Rural,13123,High,0,1,Winter,10000.0
1,1050,2023-01-02,Urban,129657,Medium,0,0,Winter,10000.0
2,1060,2023-01-03,Suburban,69842,Medium,1,0,Summer,10000.0
3,1015,2023-01-04,Suburban,36342,Medium,0,0,Winter,10000.0
4,1047,2023-01-05,Urban,68152,Low,1,0,Monsoon,10000.0


In [4]:
# Preprocessing categorical data
df['location_type'] = df['location_type'].replace({'Rural': 0, 'Suburban': 1, 'Urban': 2})
df['income_level'] = df['income_level'].replace({'Low': 0, 'Medium': 1, 'High': 2})
df['season'] = df['season'].replace({'Summer': 0, 'Monsoon': 1, 'Winter': 2})

In [5]:
# Feature engineering
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['weekday'] = df['date'].dt.weekday
df['is_weekend'] = df['weekday'].apply(lambda x: 1 if x >= 5 else 0)
df['quarter'] = df['month'] // 4 + 1
df['weekofyear'] = df['date'].dt.isocalendar().week.astype(int)

In [6]:
df = df.drop(columns=['date'])
print("Data preprocessing completed!")
df.info()

Data preprocessing completed!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atm_id                  1000 non-null   int64  
 1   location_type           1000 non-null   int64  
 2   nearby_population       1000 non-null   int64  
 3   income_level            1000 non-null   int64  
 4   holiday_flag            1000 non-null   int64  
 5   local_event_flag        1000 non-null   int64  
 6   season                  1000 non-null   int64  
 7   forecasted_cash_demand  1000 non-null   float64
 8   year                    1000 non-null   int32  
 9   month                   1000 non-null   int32  
 10  day                     1000 non-null   int32  
 11  weekday                 1000 non-null   int32  
 12  is_weekend              1000 non-null   int64  
 13  quarter                 1000 non-null   int32  
 14  weekofyear 

In [7]:
# features and target
y = df['forecasted_cash_demand']
X = df[['location_type', 'nearby_population', 'income_level',
        'holiday_flag', 'local_event_flag', 'season',
        'weekday', 'is_weekend', 'quarter', 'weekofyear']]


In [8]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2529, test_size=0.25)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (750, 10)
Test set size: (250, 10)


In [9]:
# Train models
print("Training Linear Regression...")
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

print("Training Random Forest...")
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

print("Training Gradient Boosting...")
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)

print("\nAll models trained successfully!")

Training Linear Regression...
Training Random Forest...
Training Gradient Boosting...

All models trained successfully!


In [10]:
# predictions
lr_pred = lr_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
gb_pred = gb_model.predict(X_test)

In [11]:
# Calculate metrics
models_performance = {
    'Linear Regression': {
        'model': lr_model,
        'pred': lr_pred,
        'mape': mean_absolute_percentage_error(y_test, lr_pred),
        'r2': r2_score(y_test, lr_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, lr_pred))
    },
    'Random Forest': {
        'model': rf_model,
        'pred': rf_pred,
        'mape': mean_absolute_percentage_error(y_test, rf_pred),
        'r2': r2_score(y_test, rf_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, rf_pred))
    },
    'Gradient Boosting': {
        'model': gb_model,
        'pred': gb_pred,
        'mape': mean_absolute_percentage_error(y_test, gb_pred),
        'r2': r2_score(y_test, gb_pred),
        'rmse': np.sqrt(mean_squared_error(y_test, gb_pred))
    }
}


In [12]:
# Display performance
print("Model Performance Summary:")
print("=" * 80)
for model_name, info in models_performance.items():
    print(f"\n{model_name}:")
    print(f"  MAPE: {info['mape']:.4f}")
    print(f"  R² Score: {info['r2']:.4f}")
    print(f"  RMSE: ₹{info['rmse']:,.2f}")

Model Performance Summary:

Linear Regression:
  MAPE: 0.0061
  R² Score: -0.0366
  RMSE: ₹151.27

Random Forest:
  MAPE: 0.0033
  R² Score: -0.4000
  RMSE: ₹175.80

Gradient Boosting:
  MAPE: 0.0040
  R² Score: -1.0179
  RMSE: ₹211.06


In [13]:
## Step 7: Visualization Functions
def create_feature_importance_chart():
    """Create feature importance visualization"""
    importances = rf_model.feature_importances_
    features = X.columns

    fig = go.Figure(go.Bar(
        x=importances,
        y=features,
        orientation='h',
        marker=dict(
            color=importances,
            colorscale='Blues',
            showscale=False
        )
    ))

    fig.update_layout(
        title='Feature Importance Analysis',
        xaxis_title='Importance Score',
        yaxis_title='Features',
        height=400,
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=12)
    )

    return fig


def create_model_comparison_chart():
    """Create model performance comparison chart"""
    models = list(models_performance.keys())
    r2_scores = [models_performance[m]['r2'] for m in models]
    mape_scores = [models_performance[m]['mape'] for m in models]

    fig = go.Figure()

    fig.add_trace(go.Bar(
        name='R² Score',
        x=models,
        y=r2_scores,
        marker_color='#2E86AB',
        text=[f'{v:.3f}' for v in r2_scores],
        textposition='outside'
    ))

    fig.add_trace(go.Bar(
        name='MAPE',
        x=models,
        y=mape_scores,
        marker_color='#A23B72',
        text=[f'{v:.3f}' for v in mape_scores],
        textposition='outside'
    ))

    fig.update_layout(
        title='Model Performance Comparison',
        xaxis_title='Model',
        yaxis_title='Score',
        barmode='group',
        height=400,
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=12),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
    )

    return fig


def create_prediction_gauge(prediction):
    """Create a gauge chart for the prediction"""
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=prediction,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Predicted Cash Demand (₹)"},
        gauge={
            'axis': {'range': [None, 120000]},
            'bar': {'color': "#2E86AB"},
            'steps': [
                {'range': [0, 50000], 'color': "#E8F4F8"},
                {'range': [50000, 80000], 'color': "#B8D4E0"},
                {'range': [80000, 120000], 'color': "#88B4C8"}
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 80000
            }
        }
    ))

    fig.update_layout(
        height=350,
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=14)
    )

    return fig


def create_actual_vs_predicted_chart(model_choice):
    """Create actual vs predicted scatter plot"""
    y_pred = models_performance[model_choice]['pred']

    fig = go.Figure()

    # Scatter plot
    fig.add_trace(go.Scatter(
        x=y_test,
        y=y_pred,
        mode='markers',
        marker=dict(
            size=8,
            color=y_test,
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Actual Value")
        ),
        name='Predictions'
    ))
    # prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    fig.add_trace(go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(color='red', dash='dash', width=2),
        name='Perfect Prediction'
    ))

    fig.update_layout(
        title=f'Actual vs Predicted Values - {model_choice}',
        xaxis_title='Actual Cash Demand (₹)',
        yaxis_title='Predicted Cash Demand (₹)',
        height=350,
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=12),
        showlegend=True
    )

    return fig

In [14]:
create_feature_importance_chart().show()
create_model_comparison_chart().show()
create_actual_vs_predicted_chart('Random Forest').show()

In [ ]:
## Prediction Function
def predict_cash(location_type, nearby_population, income_level,
                 holiday_flag, local_event_flag, season, weekday,
                 model_choice):

    location_map = {"Rural": 0, "Suburban": 1, "Urban": 2}
    income_map = {"Low": 0, "Medium": 1, "High": 2}
    season_map = {"Summer": 0, "Monsoon": 1, "Winter": 2}
    is_weekend = 1 if weekday >= 5 else 0
    month = 6
    quarter = month // 4 + 1
    weekofyear = 24

    input_data = np.array([[
        location_map[location_type],
        nearby_population,
        income_map[income_level],
        holiday_flag,
        local_event_flag,
        season_map[season],
        weekday,
        is_weekend,
        quarter,
        weekofyear
    ]])
    model_info = models_performance[model_choice]
    prediction = model_info['model'].predict(input_data)[0]
    if prediction > 80000:
        recommendation = "High Demand Alert"
        action = "Increase refill frequency and monitor closely"
        priority = "Critical"
        color = "#D32F2F"
    elif prediction > 50000:
        recommendation = "Moderate Demand"
        action = "Maintain regular monitoring schedule"
        priority = "Medium"
        color = "#F57C00"
    else:
        recommendation = "Normal Demand"
        action = "Standard refill schedule is sufficient"
        priority = "Low"
        color = "#388E3C"
    gauge_chart = create_prediction_gauge(prediction)
    comparison_chart = create_actual_vs_predicted_chart(model_choice)
    result_html = f"""
    <div style="padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); border-radius: 10px; color: white; margin-bottom: 20px;">
        <h2 style="margin: 0 0 10px 0;">Prediction Summary</h2>
        <h1 style="margin: 0; font-size: 2.5em;">₹{prediction:,.2f}</h1>
        <p style="margin: 5px 0 0 0; opacity: 0.9;">Forecasted Cash Demand</p>
    </div>

    <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 15px; margin-bottom: 20px;">
        <div style="padding: 15px; background: #f5f5f5; border-radius: 8px; border-left: 4px solid #2E86AB;">
            <div style="font-size: 0.9em; color: #666; margin-bottom: 5px;">MAPE</div>
            <div style="font-size: 1.5em; font-weight: bold; color: #2E86AB;">{model_info['mape']:.4f}</div>
        </div>
        <div style="padding: 15px; background: #f5f5f5; border-radius: 8px; border-left: 4px solid #A23B72;">
            <div style="font-size: 0.9em; color: #666; margin-bottom: 5px;">R² Score</div>
            <div style="font-size: 1.5em; font-weight: bold; color: #A23B72;">{model_info['r2']:.4f}</div>
        </div>
        <div style="padding: 15px; background: #f5f5f5; border-radius: 8px; border-left: 4px solid #F18F01;">
            <div style="font-size: 0.9em; color: #666; margin-bottom: 5px;">RMSE</div>
            <div style="font-size: 1.5em; font-weight: bold; color: #F18F01;">₹{model_info['rmse']:,.2f}</div>
        </div>
    </div>

    <div style="padding: 20px; background: {color}15; border-radius: 8px; border-left: 4px solid {color};">
        <h3 style="margin: 0 0 10px 0; color: {color};">Decision Support</h3>
        <div style="margin-bottom: 10px;">
            <strong>Status:</strong> {recommendation} <span style="background: {color}; color: white; padding: 3px 8px; border-radius: 3px; font-size: 0.85em; margin-left: 10px;">{priority}</span>
        </div>
        <div>
            <strong>Recommended Action:</strong> {action}
        </div>
    </div>
    """

    return result_html, gauge_chart, comparison_chart
    ## Gradio Interface
custom_css = """
.gradio-container {
    font-family: 'Inter', 'Segoe UI', sans-serif;
}

.main-header {
    text-align: center;
    padding: 30px 20px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 10px;
    margin-bottom: 30px;
}

.main-header h1 {
    margin: 0;
    font-size: 2.5em;
    font-weight: 700;
}

.main-header p {
    margin: 10px 0 0 0;
    font-size: 1.1em;
    opacity: 0.9;
}

footer {
    display: none !important;
}
"""

with gr.Blocks(css=custom_css, title="ATM Cash Demand Forecasting", theme=gr.themes.Soft()) as interface:

    gr.HTML("""
        <div class="main-header">
            <h1>ATM Cash Demand Forecasting System</h1>
            <p>Made By:Aatif Pathan(572),Akshat Kaushik(578),Ayushi Bindal(341),Yash Kataria(566) </p>
        </div>
    """)

    with gr.Tabs():
        with gr.Tab("Prediction"):
            gr.Markdown("### Input Parameters")

            with gr.Row():
                with gr.Column():
                    location_type = gr.Dropdown(
                        ["Rural", "Suburban", "Urban"],
                        label="Location Type",
                        info="ATM location category"
                    )

                    nearby_population = gr.Number(
                        label="Nearby Population",
                        info="Population within service area"
                    )

                    income_level = gr.Dropdown(
                        ["Low", "Medium", "High"],
                        label="Income Level",
                        info="Average income level of area"
                    )

                    season = gr.Dropdown(
                        ["Summer", "Monsoon", "Winter"],
                        label="Season",
                        info="Current season"
                    )

                with gr.Column():
                    holiday_flag = gr.Radio(
                        [0, 1],
                        label="Holiday",
                        info="Is today a holiday?",
                        value=0
                    )

                    local_event_flag = gr.Radio(
                        [0, 1],
                        label="Local Event",
                        info="Is there a local event?",
                        value=0
                    )

                    weekday = gr.Slider(
                        0, 6,
                        step=1,
                        label="Weekday",
                        info="0=Monday, 6=Sunday"
                    )

                    model_choice = gr.Radio(
                        ["Linear Regression", "Random Forest", "Gradient Boosting"],
                        label="Prediction Model",
                        value="Random Forest",
                        info="Select the ML model to use"
                    )

            predict_button = gr.Button(
                "Generate Forecast",
                variant="primary",
                size="lg"
            )

            gr.Markdown("---")
            gr.Markdown("### Prediction Results")

            output_html = gr.HTML()

            with gr.Row():
                gauge_plot = gr.Plot(label="Demand Gauge")
                comparison_plot = gr.Plot(label="Model Accuracy")

        with gr.Tab("Model Analytics"):
            gr.Markdown("### Model Performance Comparison")
            with gr.Row():
                model_comparison = gr.Plot(value=create_model_comparison_chart())
                feature_importance = gr.Plot(value=create_feature_importance_chart())

            gr.Markdown("### Performance Metrics Summary")

            metrics_data = []
            for model_name, info in models_performance.items():
                metrics_data.append([
                    model_name,
                    f"{info['mape']:.4f}",
                    f"{info['r2']:.4f}",
                    f"₹{info['rmse']:,.2f}"
                ])

            gr.Dataframe(
                value=metrics_data,
                headers=["Model", "MAPE", "R² Score", "RMSE"],
                label="Model Metrics"
            )

    predict_button.click(
        fn=predict_cash,
        inputs=[
            location_type,
            nearby_population,
            income_level,
            holiday_flag,
            local_event_flag,
            season,
            weekday,
            model_choice
        ],
        outputs=[output_html, gauge_plot, comparison_plot]
    )

interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://a29f262623e552849a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
